In [1]:
import numpy as np
import math
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
data_dir = './ECE-699/FGSM_EPS8/'
listdir = os.listdir(data_dir)
pos_arr = np.array([])
cnt = 0
for name in tqdm(iter(listdir)):
    cnt +=1
    if name[-3:] != 'npy':
        continue
    DCT_cof = np.load(data_dir+name)
    DC_DCT_cof = DCT_cof[:,7,7]
    pos_arr = np.concatenate((pos_arr,DC_DCT_cof), axis=0)

8it [00:00, 21.85it/s]


In [34]:
mean = np.mean(Y_n)
neg_Y_n = Y_n[Y_n<mean]-mean
pos_Y_n = Y_n[Y_n>=mean]-mean
inv_neg_Y_n = -1*(neg_Y_n)
fold_arr = np.concatenate((pos_Y_n,inv_neg_Y_n)) # abs(zero_mean(array))
fold_arr.sort()
unique_fold_arr = np.unique(fold_arr)
n_unique = len(unique_fold_arr)
n_sample = len(Y_n)

In [35]:
fold_arr

array([6.49461591e-10, 3.83525678e-09, 6.65712884e-09, ...,
       4.39286613e-02, 4.68693814e-02, 4.71016996e-02])

In [12]:
a = np.random.rand(10)
a.sort()
a

array([0.03628099, 0.10897057, 0.11174564, 0.1798898 , 0.3478096 ,
       0.44011602, 0.47674969, 0.61107218, 0.64934282, 0.76153625])

In [30]:
a = np.array([0.03628099, 0.10897057, 0.11174564, 0.1798898 , 0.3478096 ,
       0.44011602, 0.47674969, 0.47674969, 0.61107218, 0.64934282, 0.76153625])
a.sort()
a

array([0.03628099, 0.10897057, 0.11174564, 0.1798898 , 0.3478096 ,
       0.44011602, 0.47674969, 0.47674969, 0.61107218, 0.64934282,
       0.76153625])

In [31]:
np.searchsorted(a,0.47674969, side='right')

8

In [32]:
np.searchsorted(a,0.47674969, side='left')

6

In [35]:
a[8]

0.61107218

In [38]:
def LPTCM(Y_n, a, d):
    def determine_theta(Y_n, y_c):
        eps = 1e-3
        # 6. Determine theta^+(y_c) and theta^-(y_c) to be the ML estimate
        # of theta for truncated distribution
        C = np.mean(np.abs(Y_n))
        if C == 0:
            return 0
        elif C >= y_c/2:
            return float('inf')
        else:
            last_Lambda = C
        while True:
            exp = math.exp(y_c/last_Lambda)
            Lambda= C+ (y_c)/(exp-1)
            if abs(Lambda-last_Lambda) < eps:
                break
            last_Lambda = Lambda
        return Lambda
    b_lst = np.array([])
    Lambda_lst = np.array([])
    g_lst = np.array([])
    yc_lst = np.array([])
    def calc_g(mean0_abs_arr, N1_p, n_sample, Lambda, y_c ):
        # a: the largest magnitude a sample y can take
        b = N1_p/n_sample
        N1_log_likelihood = N1_p*math.log(b/(1-math.exp(-y_c/Lambda))/(2*Lambda))-np.sum(mean0_abs_arr[:N1_p])
        N2_log_likelihood = (n_sample-N1_p)*math.log((1-b)/(2*(a-y_c)))
        return N1_log_likelihood + N2_log_likelihood, b
    
    mean = np.mean(Y_n)
    neg_Y_n = Y_n[Y_n<mean]-mean
    pos_Y_n = Y_n[Y_n>=mean]-mean
    inv_neg_Y_n = -1*(neg_Y_n)
    mean0_abs_arr = np.concatenate((pos_Y_n,inv_neg_Y_n)) # abs(zero_mean(array))
    print('start sort')
    mean0_abs_arr.sort() # 1. sort {|Y_n|} in ascending order into W_1 <=...<=W_n
    print('finish sort')
    yc_lst = np.unique(Y_n>=mean)-mean
    n_sample = len(Y_n) 
    m = np.searchsorted(yc_lst, d, side='right') # 2. determine m = min{i: W_i>=d}
    n = len(yc_lst)
    i = 0
    for idx in tqdm(range(m,n)):
        y_c = yc_lst[idx]
        N1_p = np.searchsorted(mean0_abs_arr, y_c, side='right')
        print(N1_p)
        Lambda_p = determine_theta(mean0_abs_arr[:N1_p], y_c)
        g_p, B_p = calc_g(mean0_abs_arr, N1_p, n_sample, Lambda_p, y_c)
        
        g_lst = np.append(g_lst, g_p)
        b_lst = np.append(b_lst, B_p)
        Lambda_lst = np.append(Lambda_lst,Lambda_n)
        yc_lst = np.append(yc_lst,y_c)
    max_idx = np.argmax(g_lst)
    return b_lst[max_idx],Lambda_lst[max_idx],yc_lst[max_idx]

In [40]:
a = 0.061
d= 0.017
LPTCM(pos_arr,a,d)

start sort
finish sort


  0%|          | 0/1 [00:00<?, ?it/s]

3048235


ValueError: math domain error

In [9]:
def LPTCM(Y_n, a, d):
    b_lst = np.array([])
    Lambda_lst = np.array([])
    g_lst = np.array([])
    print('start sort')
    Y_n.sort() # 1. sort {|Y_n|} in ascending order into W_1 <=...<=W_n
    W_lst = np.unique(Y_n)
    print('finish sort')
    m = np.sum([W_lst<=d]) # 2. determine m = min{i: W_i>=d}
    n = len(W_lst)
    zero_idx = np.sum([Y_n<0])
    idx_choose = 0
    for idx in tqdm(range(m,n)): # 3. foreach y_c in {d, W_m, W_{m+1},...,W_n}do
        y_c = W_lst[idx]
        N1_p, N1 = np.abs(Y_n)<=y_c, np.abs(Y_n)<y_c # 4. set N_1^+(y_c) = {i: |Y_i|<=y_c}, N_1(y_c) = {i: |Y_i|<y_c}
        N2_p, N2 = np.abs(Y_n)>y_c, np.abs(Y_n)>=y_c
        B_p, B_n = np.sum(N1_p)/len(Y_n), np.sum(N1)/len(Y_n) # 5.Compute b^+(y_c) and b^-(y_c)
        Lambda_p, Lambda_n = determine_theta(N1_p, Y_n, y_c), determine_theta(N1, Y_n, y_c) # 6.
        g_p, g_n = calc_g(N1_p, N2_p, Y_n, a, B_p, y_c, Lambda_p), calc_g(N1, N2, Y_n, a, B_n, y_c, Lambda_n)
        if g_p >= g_n:
            print(Lambda_p,g_p)
            g_lst = np.append(g_lst,g_p)
            Lambda_lst = np.append(Lambda_lst,Lambda_p)
            b_lst = np.append(b_lst,B_p)
        else:
            print(Lambda_n,g_n)
            g_lst = np.append(g_lst,g_n)
            Lambda_lst = np.append(Lambda_lst,Lambda_n)
            b_lst = np.append(b_lst,B_n)
    max_idx = np.argmax(g_lst)
    def determine_theta(N1, Y_n, y_c):
        eps = 1e-3
        # 6. Determine theta^+(y_c) and theta^-(y_c) to be the ML estimate
        # of theta for truncated distribution
        C = np.mean(np.abs(Y_n[N1]))
        if C == 0:
            return 0
        elif C >= y_c/2:
            return float('inf')
        else:
            last_Lambda = C
        while True:
            exp = math.exp(y_c/last_Lambda)
            Lambda= C+ (y_c)/(exp-1)
            if abs(Lambda-last_Lambda) < eps:
                break
            last_Lambda = Lambda
        return Lambda

    def calc_g(N1, N2, Y_n, a, b, y_c, Lambda):
        # a: the largest magnitude a sample y can take
        return np.sum(N1)*math.log(b/(1-math.exp(-y_c/Lambda))*1/(2*Lambda))+Lambda*np.sum(-np.abs(Y_n[N1])) + np.sum(N2)*(1-b)/(2*(a-y_c))

    return b_lst[max_idx], Lambda_lst[max_idx]

In [10]:
print(LPTCM(pos_arr, a = 0.061, d= 0.017))


start sort
finish sort


  0%|          | 0/1635 [00:00<?, ?it/s]


UnboundLocalError: local variable 'determine_theta' referenced before assignment

In [4]:
pwd

'/home/deponce/ECE699'